In [ ]:
def DCGALL(base_input_dir, base_output_dir, dataset_dir, N=20):
    for fold in range(1, 6):
        print(f"\n🔄 Processing Fold {fold}...")

        # Load calTest dan convert ke list ground truth
        testdata = load_Dataset(dataset_dir, fold)
        dataTestUniq = uniqDataTest(testdata)

        fold_input_path = os.path.join(base_input_dir, str(fold))
        fold_output_path = os.path.join(base_output_dir, str(fold))
        os.makedirs(fold_output_path, exist_ok=True)

        # Loop semua file .joblib di folder fold ini
        for filename in os.listdir(fold_input_path):
            if filename.endswith(".joblib"):
                filepath = os.path.join(fold_input_path, filename)

                try:
                    print(f"   Processing: {filename}")
                    openTopN = joblib.load(filepath)
                    calDCGN = dcgN2(dataTestUniq, openTopN, N)

                    # Simpan hasil DCGN
                    result_name = filename.replace(".joblib", "_dcg.joblib")
                    result_path = os.path.join(fold_output_path, result_name)
                    # joblib.dump(calDCGN, result_path)
                    # print(f"     Saved to: {result_path}")

                    # Simpan hasil to execel
                    excel_name = filename.replace(".joblib", "_dcg.xlsx")
                    excel_path = os.path.join(fold_output_path, excel_name)
                    calDCGN.to_excel(excel_path)

                except Exception as e:
                    print(f"   Error processing {filename}: {e}")

#### JACCARD 

In [ ]:
# calProses = DCGALL(
#     "../case/topN/Jac/topNUser", 
#     "../case/DCG/Jac/user", 
#     "../ml-100k",
#     N=20
# )

In [ ]:
# calProses = DCGALL(
#     "../case/topN/Jac/topNItem", 
#     "../case/DCG/Jac/item", 
#     "../ml-100k",
#     N=20
# )

#### RJ 

In [ ]:
# calProsesRJ = DCGALL(
#     "../case/topN/RJ/topNUser", 
#     "../case/DCG/RJ/user", 
#     "../ml-100k",
#     N=20
# )

In [ ]:
# calProsesRJ = DCGALL(
#     "../case/topN/RJ/topNItem", 
#     "../case/DCG/RJ/item", 
#     "../ml-100k",
#     N=20
# )

#### HYBRID  

In [ ]:
# calProses = DCGALL(
#     "../case/topN/Jac/topNHybrid", 
#     "../case/DCG/Jac/hybrid", 
#     "../ml-100k",
#     N=20
# )

In [ ]:
# calProses = DCGALL(
#     "../case/topN/RJ/topNHybrid", 
#     "../case/DCG/RJ/hybrid", 
#     "../ml-100k",
#     N=20
# )

#### HYBRID TERBAIK 

In [ ]:
# calProses = DCGALL(
#     "../case/topN/Jac/topNhybridTerbaik", 
#     "../case/DCG/Jac/hybridTerbaik", 
#     "../ml-100k",
#     N=20
# )

In [ ]:
# calProses = DCGALL(
#     "../case/topN/RJ/topNhybridTerbaik", 
#     "../case/DCG/RJ/hybridTerbaik", 
#     "../ml-100k",
#     N=20
# )

In [ ]:
import pandas as pd
import joblib

# Parameter
values1 = 5
values2 = 10
values3 = 1

# Untuk menyimpan hasil rata-rata tiap fold
all_avg_rows = []

# Loop untuk 5 fold
for fold in range(1, 6):
    filepath = f"../case/DCG/Jac/hybridTerbaik/{fold}/{values1}_{values2}_{values3}_dcg.joblib"
    df = joblib.load(filepath)
    
    # Ambil baris 'Rata-rata'
    avg_row = df.loc["Rata-rata"].copy()
    
    # Simpan info fold
    avg_row["Fold"] = f"Fold {fold}"
    
    # Hitung rata-rata across DCG columns
    dcg_cols = [col for col in avg_row.index if col.startswith("DCG@")]
    avg_row["Avg"] = avg_row[dcg_cols].astype(float).mean()
    
    all_avg_rows.append(avg_row)

# Konversi ke DataFrame
final_df = pd.DataFrame(all_avg_rows)

# Hitung rata-rata akhir untuk semua fold (kolom DCG dan kolom "Avg")
numeric_cols = [col for col in final_df.columns if col.startswith("DCG@")] + ["Avg"]
value_cols = final_df[numeric_cols].astype(float)
mean_row = value_cols.mean().to_dict()
mean_row["Fold"] = "Rata-rata"

# Tambahkan ke DataFrame
final_df = pd.concat([final_df, pd.DataFrame([mean_row])], ignore_index=True)

# Atur ulang kolom agar "Fold" di depan
cols = ["Fold"] + [col for col in final_df.columns if col != "Fold"]
final_df = final_df[cols]

# fold mana yang terbaik
best_fold = final_df.loc[final_df["Avg"].idxmax()]
print(f"Fold terbaik: {best_fold['Fold']}, dengan rata-rata DCG: {best_fold['Avg']}")


# Tampilkan hasil
final_df
